## Аугментация данных

Аугментация данных - это методика создания дополнительных данных из уже имеющихся данных. Наиболее популярными способами аугментации изображений являются:

* отображение по вертикали или горизонтали (flipping)
* поворот изображения на определенный угол (rotation)
* создание отступа (padding)
* вырезание части изображения (cropping)
* добавление шума (adding noise)
* манипуляции с цветом (color jittering)

Воспользуемся методом **RandomZoom**, который позволяет приближать случайные участки изображения.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from keras.models import Sequential # импортируем пустую последовательную модель Sequential, на которую последовательно навешиваем слои 
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, InputLayer, RandomZoom # импортируем слои Dense (полносвязный), Flatten (чтобы расплющить результаты, сделать их одномерным массивом; чтобы слои Dense смогли принять данные с свёрточных слоёв); слой Dropout позволяет избежать переобучения
from keras.datasets import fashion_mnist # импортируем из модуля keras.datasets класс fashion_mnist (база данных изображений одежды)
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

import requests
from PIL import Image
from io import BytesIO

%matplotlib inline

2023-01-26 21:02:18.845648: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Импортируем данные

In [2]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data() # x_train, x_test: массив данных изображений в градациях серого uint8 с формой (размерность 28х28). y_train, y_test: массив меток uint8 (целые числа в диапазоне от 0 до 9)

## Нормализация данных

In [3]:
x_train_norm = x_train.astype('float32') / 255 # приводим данные к вещественным числам и делим на 255, потому что нейросети удобнее работать со значениями от 0 до 1, а не от 0 до 255
x_test_norm = x_test.astype('float32') / 255

In [4]:
x_train_norm[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [5]:
y_train_norm = np_utils.to_categorical(y_train, 10) # проводим one-hot кодирование для labels (y) для 10 классов (10 классов картинок в датасете), чтобы нейросеть предсказывала данные в правильном формате (на выходе у сети будет 10 нейронов)
y_test_norm = np_utils.to_categorical(y_test, 10)

In [6]:
y_train_norm[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [7]:
x_train_norm.shape

(60000, 28, 28)

In [8]:
x_train_norm.shape[1]

28

## Создание модели

In [21]:
model = Sequential()
model.add(Conv2D(32, kernel_size=3, activation='relu', padding='same', input_shape=(x_train_norm.shape[1], x_train_norm.shape[2], 1))) # добавляем свёрточный слой Conv2D; количество фильтров - 32; kernel_size - размер фильтра - 3х3; добавляем нелинейную активацию (функцию) Relu, чтобы всю нейросеть нельзя было свернуть в один нейрон; в самый первый слой сети передаём параметр input_shape=(28, 28) - размерность данных, которые будут подаваться в нейросеть - берётся из x_train_norm.shape без первого показтеля (количество экземпляров)
model.add(MaxPooling2D(pool_size=2)) # добавляем слой Pooling с параметром pool_size - размер фильтров слоя Pooling
model.add(Dropout(0.25)) # слой Dropout служит для того, чтобы модель не переобучалась; его смысл в том, что каждую итерацию обучения он случайным образом выключает какие-то из нейронов; 0.25 - доля тех нейронов, которая будет на время выключаться из процесса обучения; это связано с тем, что если моедль окажется зависима от какого-то одного нейрона (переобучится на нём), то когда этот нейрон попадёт в те 25%, что мы задали, нейросети придётся обучать модель на других нейронах

model.add(Flatten()) # преобразовываем данные в одномерный массив (одномерный вектор) для того, чтобы можно было работать со слоем Dense
model.add(Dense(32, activation='relu')) # полносвязный слой Dense не может работать с двумерным массивом данных (ему нужен одномерный массив), поэтому приходится предобрабатывать данные с помощью слоя Flatten
model.add(Dense(10, activation='sigmoid')) # последний выходной слой (тоже полносвязный), однако в нём количество нейронов будет равно количеству классов датафрейма (10); softmax - специальная активация, при которой выходные числа преобразовываются в формат процентов (%)

2023-01-26 21:08:52.486507: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # скомпилируем модель - зададим правила, по которым модель будет обучаться 

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 32)                200736    
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                        

In [9]:
type(x_train_norm)

numpy.ndarray

In [10]:
x_train_float = x_train_norm.astype('float32')

In [11]:
type(x_train_float)

numpy.ndarray

In [12]:
x_train_float = x_train_float.reshape(-1, 1)
x_train_float

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [13]:
x_train_float.shape

(47040000, 1)

In [14]:
x_test_float = x_test_norm.reshape(-1, 1)
x_test_float

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [15]:
x_test.shape

(10000, 28, 28)

In [16]:
x_train_aug = x_train_float.reshape(-1, 28, 28, 1)
x_test_aug = x_test_float.reshape(-1, 28, 28, 1)
y_train_aug = np.reshape(y_train, (60000, 1, 1, 1))
y_test_aug = np.reshape(y_test, (10000, 1, 1, 1))
print("x_train shape: ", x_train_aug.shape)
print("x_test shape: ", x_test_aug.shape)
print("y_train shape: ", y_train_aug.shape)
print("y_test shape: ", y_test_aug.shape)

x_train shape:  (60000, 28, 28, 1)
x_test shape:  (10000, 28, 28, 1)
y_train shape:  (60000, 1, 1, 1)
y_test shape:  (10000, 1, 1, 1)


In [17]:
datagen = ImageDataGenerator(
    rotation_range = 8,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.1, # Randomly zoom image 
    shear_range = 0.3,# shear angle in counter-clockwise direction in degrees  
    width_shift_range=0.08,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.08,  # randomly shift images vertically (fraction of total height)
    vertical_flip=True)  # randomly flip images

In [18]:
datagen.fit(x_train_aug)

In [24]:
epochs = 10
batch_size = 86

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [25]:
history = model.fit_generator(datagen.flow(x_train_aug, y_train, batch_size = batch_size),
                              epochs = epochs, validation_data = (x_test, y_test),
                              verbose = 2, steps_per_epoch = x_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

/var/folders/hy/pfmkjt294bl40vdb0nfc_y4c0000gn/T/ipykernel_48035/1180655994.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(x_train_aug, y_train, batch_size = batch_size),


Epoch 1/10


ValueError: in user code:

    File "/Users/andrey/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/andrey/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/andrey/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/andrey/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/andrey/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "/Users/andrey/opt/anaconda3/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 265, in